In [ ]:
import numpy as np

In [ ]:
from pprint import pprint

In [ ]:
import json
import vtk

In [ ]:
from matplotlib import pyplot
import matplotlib
pyplot.ion()

In [ ]:
with open('/tmp/paths.json') as f:
    data = f.read()

In [ ]:
paths = map(str.strip, filter(lambda s: s, data.split("\n-----------------------------------------\n")))
paths = list(map(lambda s: json.loads(s), paths))

In [ ]:
for p in paths:
    p['max_contribution'] = max(*p['contribution']['pixel_value'])

In [ ]:
len(paths)

In [ ]:
def HasTotalReflect(wi, n, eta):
    cosThetaI = np.dot(n, wi)
    sin2ThetaI = max(0, 1- cosThetaI*cosThetaI)
    sin2ThetaI *= eta*eta
    return sin2ThetaI >= 1

In [ ]:
points = vtk.vtkPoints()
lines  = vtk.vtkCellArray()
idx = 0
for p in paths:
    for i in range(len(p)-1):
        points.InsertNextPoint(p[i]['position'])
        points.InsertNextPoint(p[i+1]['position'])
        line = vtk.vtkLine()
        line.GetPointIds().SetId(0, idx)
        line.GetPointIds().SetId(1, idx+1)
        lines.InsertNextCell(line)
        idx += 2

polydata = vtk.vtkPolyData()
polydata.SetPoints(points)
polydata.SetLines(lines)
polydata.Modified()
polydata.Update()

writer = vtk.vtkXMLPolyDataWriter()
writer.SetFileName('/tmp/lines.vtp')
writer.SetInput(polydata)
writer.Write()